In [1]:
# Loading libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from IPython.display import Math, Latex, display

In [2]:
# reading the data files

coad_data = pd.read_csv('/content/coad_data.csv',index_col=0)
cov = pd.read_csv('/content/clinical_data.csv',index_col=0)
coad_data.shape

(890, 508)

In [ ]:
# meta data table
cov

,sample_type,stage,age,gender
TCGA-AA-A00K-01A-02R-A002-07,Primary Tumor,Stage IIA,79,male
TCGA-D5-5538-01A-01R-1653-07,Primary Tumor,Stage IIIB,60,female
TCGA-AA-A00A-01A-01R-A002-07,Primary Tumor,Stage IIA,80,male
TCGA-AA-A02K-01A-03R-A32Y-07,Primary Tumor,Stage IV,50,male
TCGA-D5-6922-01A-11R-1928-07,Primary Tumor,Stage IIIA,76,male
...,...,...,...,...
TCGA-AA-A00D-01A-01R-A002-07,Primary Tumor,Stage I,70,male
TCGA-AA-A00U-01A-01R-A002-07,Primary Tumor,Stage IIIB,50,male
TCGA-AA-A01V-01A-23R-A083-07,Primary Tumor,Stage I,59,male
TCGA-AA-3519-01A-02R-0821-07,Primary Tumor,Stage III,63,male


# Recommendation System

## Gene of interest calculation

In [ ]:
def recommendation(data):

  # converting the data into a matrix
  dataset = np.matrix(data)

  # calcualting the similarity matrix
  def similarity_matrix(x):

    # considering the number of genes as the number of rows
    genes = x.shape[0]

    # creating a matrix of zeros
    mat = np.zeros((genes,genes))

    # for every row and each column calculating the similarity
    for i in range(genes):
      for j in range(genes):

        similarity = x[i,j] / ((x[i,i] + x[j,j]) - x[i,j])

        # replacing the zeros in the matrix to the similarity score
        mat[i,j] = similarity

    return mat

  # calculating single single network by multiplying the data by itself
  gene_single_network = dataset @ dataset.transpose()


  # calculating similarity matrix using the above definition
  sim_mat = similarity_matrix(gene_single_network)

  # calculating the gene of interest by multiplying the similarity matrix with the data and sorking to rank them
  gene_interest = sim_mat @ dataset
  gene_ranking = np.sort(np.sum(gene_interest, axis=1))

  sorted_indices = np.argsort(gene_interest, axis=0)

  # Get the ranks for each row
  ranks = np.argsort(sorted_indices, axis=0) + 1

  # converting the ranks into a data frame
  ranks = pd.DataFrame(ranks)

  ranks.columns = data.columns
  ranks.set_index(data.index, inplace = True )

  # for each column in the ranks taking only the top 10 genes of interest
  rec = {}
  for i in range(len(ranks.columns)):
    data_sample = pd.DataFrame(ranks.iloc[:,i])
    top = data_sample.sort_values(by=ranks.columns[i])[:10]
    rec[ranks.columns[i]] = top.index

  return rec





In [ ]:
# getting the top genes for one sample in the coad data
rec = recommendation(pd.DataFrame(coad_data[coad_data.columns[2]]))

In [ ]:
rec

{'TCGA-D5-5538-01A-01R-1653-07': Index(['ENSG00000166126.11', 'ENSG00000156381.9', 'ENSG00000149809.16',
        'ENSG00000107833.10', 'ENSG00000138031.14', 'ENSG00000161888.11',
        'ENSG00000012171.20', 'ENSG00000135540.11', 'ENSG00000143248.13',
        'ENSG00000125144.14'],
       dtype='object')}

# LOGISTIC REGRESSION

## Pre Processing

In [4]:
# pre-processing the coad data by transposing it and merging with the metadata

ml_data = coad_data.transpose()
ml_data = ml_data.merge(cov,left_index=True, right_index = True)
ml_data.head()

,ENSG00000001617.12,ENSG00000002726.21,ENSG00000006015.18,ENSG00000006327.14,ENSG00000006625.18,ENSG00000006704.11,ENSG00000007306.15,ENSG00000008323.15,ENSG00000011426.11,ENSG00000011465.18,...,ENSG00000275234.1,ENSG00000276023.5,ENSG00000277443.3,ENSG00000278535.5,ENSG00000280206.1,ENSG00000284753.2,sample_type,stage,age,gender
TCGA-AA-A00K-01A-02R-A002-07,49,201,56,73,87,37,364,41,21,12,...,46,17,105,17,34,54,Primary Tumor,Stage IIA,79,male
TCGA-D5-5538-01A-01R-1653-07,50,181,38,71,65,37,41,36,24,81,...,50,28,168,7,29,25,Primary Tumor,Stage IIIB,60,female
TCGA-AA-A00A-01A-01R-A002-07,48,97,67,82,72,35,67,40,17,36,...,59,29,90,16,21,40,Primary Tumor,Stage IIA,80,male
TCGA-AA-A02K-01A-03R-A32Y-07,9,97,44,87,126,50,77,47,36,8,...,76,28,127,24,31,22,Primary Tumor,Stage IV,50,male
TCGA-D5-6922-01A-11R-1928-07,40,247,36,148,92,77,111,87,47,80,...,56,24,207,21,44,27,Primary Tumor,Stage IIIA,76,male


In [5]:
# converting the charector variables to numerical indexes

ml_data['gender'] = np.where(ml_data['gender'] == "male", 1, 0)
ml_data['sample_type'] = np.where(ml_data['sample_type'] == "Primary Tumor", 1, 0)

stage = pd.get_dummies(ml_data['stage'], drop_first=True)

ml_data.drop(['stage'], axis = 1, inplace = True)
ml_data = ml_data.merge(stage, right_index = True, left_index=True)

In [6]:
# adding a column in the 1st position with all 1 to correspond to the zero th weight vector position
ml_data = pd.DataFrame(np.ones(ml_data.shape[0]), columns=['index'],  index=ml_data.index).merge(ml_data, right_index=True,left_index=True)

In [7]:
# dividing the data into X and y

X = ml_data.drop(['sample_type'], axis = 1)
y = ml_data['sample_type']

In [8]:
# splitting the data to include 80% of the data in the trainig set and 20% in the test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train.shape

(406, 905)

In [10]:
y_train.shape

(406,)

In [11]:
# creating an random w vector to start the analysis with
w = np.random.rand(X_train.shape[1],)

In [12]:
w = [i for i in w]

In [ ]:
w[:20]

[0.38327700977653023,
 0.6749171378972578,
 0.27571357855495093,
 0.11920551932191825,
 0.9917943044632134,
 0.5279909054690984,
 0.2865083475206597,
 0.5726369746408285,
 0.2718323293657151,
 0.6687075232905306,
 0.31313297790428296,
 0.9610235504245612,
 0.04269239474855102,
 0.7216700621827405,
 0.585831962541151,
 0.38466243597548255,
 0.6216082473827589,
 0.3872593853843257,
 0.18368693395165236,
 0.28834543905270926]

## Logistic regression model

In [13]:

def multiplication(x,w):
  return w @ x.transpose()

def acitivation_function(x,w):
  return 1/(1 + np.exp(-(multiplication(x,w))))

def prediction(x,w, threshold):
  return np.where(acitivation_function(x,w) > threshold,1,0)

def loss(y,x, w,reg_rate):
  return(-1 * (np.sum(y * np.log(acitivation_function(x,w)) +
   (1-y) * np.log(acitivation_function(x,w))))+ reg_rate * np.dot(np.transpose(w),w))

def optimization(x,y,w,reg_rate):
  return(x.transpose() @ (acitivation_function(x,w) - y) + reg_rate * w)


In [14]:
def logisticregression(x,w, threshold):

  return prediction(x,w,threshold)


In [15]:
# Prediction over the train data
pred = prediction(X_train,w,0.5)

In [16]:
# Predicting the test data
pred_test = prediction(X_test, w, 0.5)

In [17]:
# viewing test data . The model is not predicting class 0 correctly
pred_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [18]:
from sklearn.metrics import confusion_matrix, classification_report

In [19]:
confusion_matrix(y_test, pred_test)

array([[ 0,  9],
       [ 0, 93]])

In [20]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.91      1.00      0.95        93

    accuracy                           0.91       102
   macro avg       0.46      0.50      0.48       102
weighted avg       0.83      0.91      0.87       102



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
rmse = np.sum((y_test - pred_test)**2)

In [22]:
rmse

9

In [23]:
# updating weight vector over 100 iterations with regularization rate as 0.0001 and learing rate as 0.01

reg_rate = 0.0001
lr = 0.01

num_epochs = 100

def weight_updates(x,y, num_epochs, lr, reg_rate):

  # creating a weight vector of all zeros
  w_vector = np.zeros(x.shape[1])


  w_all = []
  error_all = []

  # from 0 to 100 epochs performing optimization, calculating the loss function and
  # updating the vector based on the optimization and learning rate

  for i in np.arange(0,num_epochs):

    gd = optimization(x,y,w_vector,reg_rate)

    w_all.append(w_vector)

    error_all.append(loss(y,x,w_vector,reg_rate))


    grad = optimization(x,y,w_vector,reg_rate)

    w_vector = w_vector - lr*grad


  return w_vector




In [24]:
# getting the new weight vector

w_new = weight_updates(X_train,y_train, num_epochs, lr, reg_rate)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [25]:
w_new

index                   2.409765
ENSG00000001617.12    117.268758
ENSG00000002726.21   -422.621942
ENSG00000006015.18    163.899247
ENSG00000006327.14    464.845387
                         ...    
Stage IIIB              0.534949
Stage IIIC              0.354966
Stage IV                0.109989
Stage IVA               0.034997
Stage IVB               0.005000
Length: 905, dtype: float64

In [26]:
# predicting test scores based on the new weight vector

pred_update = prediction(X_test,w_new,0.5)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [27]:
# calculating the evaluation metrics for the test data using the new weight vector

rmse = np.sum(y_test - pred_update)**2

In [28]:
rmse

0

In [31]:
confusion_matrix(y_test, pred_update)

array([[ 9,  0],
       [ 0, 93]])

In [30]:
print(classification_report(y_test, pred_update))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        93

    accuracy                           1.00       102
   macro avg       1.00      1.00      1.00       102
weighted avg       1.00      1.00      1.00       102

